In [ ]:
#20240514

In [ ]:
Optimizer 클래스
▪ 매개변수 갱신을 위한 기반 클래스
▪ 구체적인 최적화 기법은 Optimizer 클래스를 상속한 자식 클래스에서 구현함
▪ 초기화 메서드 target과 hooks 라는 두 개의 인스턴스 변수를 초기화
▪ setup 메소드는 매개변수를 갖는 클래스를 인스턴스 변수인 target으로 설정
▪ update 메서드는 모든 매개변수를 갱신
▪ 구체적인 매개변수 갱신은 update_one 메서드에서 수행, 자식 클래스에서 재정의
▪ 전처리는 add_hook 메서드를 사용하여 전처리 수행

In [ ]:
class Optimizer:
    def __init__(self):
        self.target = None
        self.hooks = []

    def setup(self, target):
        self.target = target
        return self

    def update(self):
        params = [p for p in self.target.params() if p.grad is not None]
        for f in self.hooks:
            f(params)

        for param in params:
            self.update_one(param)

    def update_one(self, param):
        raise NotImplementedError()

    def add_hook(self,f):
        self.hooks.append(f)

In [ ]:
SGD 클래스 구현
▪ 경사하강법으로 매개변수를 갱신하는 클래스를 구현
▪ SGD 클래스는 Optimizer 클래스를 상속
▪ __init__ 메서드는 학습률을 받아 최기화
▪ update_one 메서드에서 매개변수 갱신 코드를 구현함

In [ ]:
class SGD(Optimizer):
    def __init__(self, lr=0.01):
        super().__init__()
        self.lr = lr

    def update_one(self, param):
        param.data -= self.lr * param.grad.data


In [ ]:
 SGD 클래스를 사용하여 회귀 문제 풀기
▪ MLP 클래스를 사용하여 모델을 생성
▪ SGD 클래스로 매개변수를 갱신 – optimizer.update( )


In [ ]:
[SGD 이외의 최적화 기법]
기울기를 이용한 최적화 기법
▪ Momentum, AdaGrad, AdaDelta, Adam
▪ Optimizer 클래스를 이용해 다양한 최적화 기법을 필요에 따라 손쉽게 전환
▪ Optimizer 클래스를 상속하여 다양한 최적화 기법을 구현

Momentum 기법
▪ W는 갱신할 가중치 매개변수, 𝜕𝐿/𝜕𝑊은 기울기, η 는 학습률
▪ v는 물리에서 말하는 속도에 해당한다.
▪ αv 는 물체가 아무런 힘을 받지 않을때 서서히 감속시키는 역할을 한다.
    v<-av-η(𝜕𝐿/𝜕𝑊)
    W<- W+v 이다.

 MomentumSGD 구현 코드
▪ 속도에 해당하는 데이터, 딕셔너리 타입의 인스턴스 변수 self.vs에 유지
▪ 초기화 시에는 vs에 아무것도 담겨있지 않음
▪ Update_one() 이 처음 호출될 때 매개변수와 같은 타입의 데이터를 생성
▪ 구현한 학습 코드에서 손쉽게 Momentum으로 전환
• optimizer = MomentumSGD(lr) 변경하면 된다.


In [ ]:
[소프트맥스 함수와 교차 엔트로피오차]
 
<다중 클래스(Multi-class Classification) 분류>
▪ 여러 클래스로 분류하는 문제
▪ 분류 대상이 여러 가지 클래스 중 어디에 속하는지 추정함.

[슬라이스 조작 함수]

<get_item 함수>
▪ Variable의 다차원 배열 중에서 일부를 슬라이스하여 뽑아줌
▪ (2, 3) 형상의 x에서 1번째 행의 원소를 추출함
▪ DeZero 함수로 구현했기 때문에 역전파도 제대로 수행

<역전파>
▪ y.backward( ) 를 호출하여 역전파 수행
▪ 슬라이스로 인한 계산은 다차원 배열의
데이터 일부를 수정하지 않고 전달
• 원래의 다차원 배열에서 데이터가 추출된
위치에 행당 기울기를 설정
• 그 외에는 0으로 설정

+++코드+++
import numpy as np
from dezero import Variable
import dezero.functions as F

x = Variable(np.array([[1,2,3],[4,5,6]]))
y = F.get_item(x, 1)
print(y)

y.backward()
print(x.grad)

In [ ]:
[슬라이스 조작 함수]

<슬라이스>
▪ 다차원 배열의 일부를 추출하는 작업 
▪ get_item 함수 사용시 같은 인덱스를 반복 지정하여 동일한 원소를 여러 번 빼낼 수 있음

x = Variable(np.array([[1,2,3],[4,5,6]]))
indices = np.array([0,0,1])
y = F.get_item(x, indices)
print(y)

<특수 메서드로 설정>
▪ get_item 함수를 Variable 의 메서드로도 사용함. 
▪ 슬라이스 작업의 역전파도 이루어짐.

[소프트맥스 함수]

<신경망으로 다중 클래스 분류>
▪ 선형 회귀 때 이용한 신경망을 그대로 사용할 수 있음
▪ MLP 클래스로 구현해둔 신경망을 그대로 이용할 수 있음
▪ 입력 데이터의 차원 수가 2이고 3개의 클래스를 분류하는 문제
+++코드+++
from dezero.models import MLP
model = MLP((10,3))

▪ 2층으로 이루어진 완전연결 신경망을 만들어 줌
▪ 첫 번째 완전연결 계층의 출력 크기는 10, 두 번째 완전연결계층의 출력 크기는 3
▪ model 은 입력 데이터를 3차원 벡터로 변환
+++코드+++
x = np.array([[0.2, -0.4]])
y = model(x)
print(y)

▪ x의 형상은 (1, 2), 샘플 데이터가 하나 있고 그 데이터는 원소가 2개인 2차원 벡터
▪ 신경망의 출력 형태는 (1, 3), 하나의 샘플 데이터가 3개의 원소로 변환
▪ (0번, 1번, 2번 원소 중) 2번 원소가 0.92401356 으로 가장 크기 때문에 2번 클래스로 분류

<소프트맥스 함수>
▪ 신경망 출력은 단순한 수치인데 이 수치를 확률로 변환하기
▪ 소프트맥스 함수의 입력 𝑦𝑘 가 총 n개라고 가정할때 K 번째 출력 𝑝𝑘 를 구하는 계산식을 이용.
▪ 분자는 입력 𝑦𝑘 의 지수 함수고, 분모는 모든 입력 지수 함수의 총합 ( 0 <= 𝑝𝑖 <= 1).
▪ p의 각 원소는 0 이상 1 이하이고, 총합은 1이 됨, 신경망의 출력을 확률로 변환

+++코드+++
from dezero import Variable, as_variable
import dezero.functions as F

def softmax1d(x):
    x = as_variable(x)
    y = F.exp()
    sum_y = F.sum(y)
    return y / sum_y


In [ ]:
model - MLP((10,3))
x = Variable(np.array([[0.2, -0.4]]))
y = model(x)
p = softmax1d(y)
print(y)
print(p)

In [1]:
▪ 샘플 데이터 각각에 소프트맥스 함수를 적용하는 경우 배치(batch) 데이터에도 소프트맥수 함수 적용

<배치 데이터를 처리하는 소프트맥스 함수>
▪ 인수 x는 2차원 데이터로 가정
▪ axis = 1 (행축)
▪ keepdims=True 이므로 각 행에서 나눗셈
▪ 더 나은 구현 방식은 Function 클래스를
상속하여 Softmax 클래스를 구현
▪ 파이썬 함수로 softmax를 구현하는 것

+++코드+++
def softmax_simple(x, axis=1)
    x = as_variable(x)
    y = exp(x)
    sum_y = sum(y, axis=axis, keepdims=True)
    return y / sum_y

SyntaxError: invalid character '▪' (U+25AA) (812313216.py, line 1)

In [ ]:
<교차 엔트로피 오차 (Cross Entropy Error)>
             
▪ 다중 클래스 분류에 적합한 손실 함수
▪ 정답 데이터의 원소는 정답에 해당하는 클래스면 1로, 그렇지 않으면 0으로 기록한다.
▪ 이러한 표현 방식을 원핫 벡터 (onehot vector)라 부름.
▪ 𝑝𝑘 는 신경망에서 소프트맥스 함수를 적용한 후의 출력한다.
▪ 정답 클래스에 해당하는 번호의 확률 p를 추출함으로써 교차 엔트로피의 오차를 계산한다.
▪ P[t]는 벡터 p에서 t번째 요소만을 추출한다.

In [ ]:
<교차 엔트로피 오차 구현>

▪ 인수 x는 신경망에서 소프트맥스 적용하기 전의 출력, t는 정답 데이터
▪ clip 함수는 x_min 이하면 x_min 으로 변환하고, x_max 이상이면 x_max로 변환
▪ np_arrange(N)은 [0, 1, …, N-1] 형태의 ndarray 인스턴스를 생성해 줌

+++코드+++
def softmax_cross_entropy_simple(x, t):
    x, t = as_variable(x), as_variable(t)
    N = x.shape(0)
    p = softmax(x)
    p = clip(p, 1e-15, 1.0)
    log_p = log(p)
    tlog_p = log_p[np.arange(N). t.data]
    y = -1 * sum(tlog_p) / N
    return y

▪ x와 정답 데이터 t를 가지고 교차 엔트로피 오차를 계산

+++코드+++
x = np.array([[0.2, -0.4],[0.3, 0.5],[1.3, -3.2],[2.1, 0.3]])
t = np.array([2, 0, 1, 0])
y = model(x)
loss = F.softmax_cross_entropy_simple(y,t)
print(loss)

In [ ]:
[다중 클래스 분류]

<다중 클래스 분류 수행>
▪ 소프트맥스 함수와 교차 엔트로피 오차를 구현한다.
▪ 스파이럴 데이터셋 이라는 작은 데이터셋을 사용하여 다중 클래스 분류 실제 수행한다.
(스파이럴은 나선형 혹은 소용돌이 모양이라는 뜻 )

<스파이럴 데이터셋>
▪ get_spiral 함수를 사용하여 스파이럴 데이터셋을 읽어오기.

+++코드+++
import dezero.datasets as ds

x, t = ds.get_spiral(train = True)
print(x.shape)
print(y.shape)

print(x[10], t[10])
print(x[110], t[110])

▪ train=True 면 학습용 데이터를 반환, False 면 테스트용 데이터를 반환
▪ 반환되는 값은 입력 데이터인 x와 정답 데이터인 t 임
▪ x와 t는 모두 ndarray 인스턴스이며, 형상은 각각 (300, 2) 와 (300, ) 임
▪ 문제는 3클래스 분류이므로 t이 원소는 0, 1, 2 중 하나가 됨

In [ ]:
<학습 코드>
▪ (1) 하이퍼파라미터 설정 – 은닉층 수와 학습률
▪ (2) 데이터 셋을 읽고 모델과 옵티마이저를 생성
        -max_epoch = 300, 미니배치로 배치 크기는 30으로 설정
▪ (3) np.random.permutation 함수를 사용하여 데이터 셋의 인덱스를 무작위로 섞음
            (무작위로 정렬된 색인 리스트를 새로 생성)
▪ (4) 미니배치 생성
            ( 미니배치의 인덱스는 방금 생성한Index에서 앞에서부터 차례로 꺼내 사용)
▪ (5) 기울기를 구하고 매개변수 갱신
▪ (6) 에포크마다 손실 함수 결과 출력

+++코드+++
import math
import numpy as np
import matplotlib.pyplot as plt
import dezero
from dezero import optimizers
import dezero.functions as F
from dezero.models import MLP

#(1)하이퍼파라미터설정
max_epoch = 300
batch_size = 30
hidden_size = 10
lr = 1.0

#(2) 데이터 읽기, 모델과 옵티마이저 생성
x, t = dezero.datasets.get_spiral(train=True)
model = MLP((hidden_size, 3))
optimizer = optimizers.SGD(lr).setup(model)


data_size = len(x)
max_iter = math.ceil(data_size / batch_size)

for epoch in range (max_epoch):
#(3)데이터 셋을 읽고 모델과 옵티마이저를 생성
    index = np.random.permutation(data_size)
    sum_loss = 0

    for i in range(max_iter):
        #(4)미니배치 생성
        batch_index = index[i * batch_size:(i + 1) * batch_size]
        batch_x = x[batch_index]
        batch_t = t[batch_index]

        #(5)기울기를 구하고 매개변수 갱신
        y = model(batch_x)
        loss = F.softmax_cross_entropy(y, batch_t)
        model.cleargrads()
        loss.backward()
        optimizer.update()
        sum_loss += float(loss.data) * len(batch_t)

        #(6)에포크마다 손실 함수 결과 출력
        avg_loss = sum_loss / dta_size
        print('epoch %d, loss %.2f' % (epoch + 1, avg_loss))

+++이 이후부터는 코드 생략 

In [ ]:
<코드 실행 후 손실 그래프 생성>
▪ 학습을 진행할수록 손실이 줄어들어듬을 확인.
▪ 학습이 완료된 신경망의 클래스 영역에 대한 결정 경계 시각화

<대규모 데이터셋의 필요성>
▪ 스파이럴 데이터셋은 작은 데이터셋이라서 ndarray 인스턴스 하나로 처리
▪ 데이터가 100만 개라면?
• 거대한 데이터를 하나의 ndarray 인스턴스로 처리하면 모든 원소를 한꺼번에 메모리에 올려야만 함
▪ 대규모 데이터를 처리할 수 있도록 데이터셋 전용 클래스인 Dataset 클래스를 만듬
▪ Dataset 클래스에는 데이터를 전처리할 수 있는 구조도 추가

In [ ]:
<Dataset 클래스 구현>
▪ Dataset 클래스는 기반 클래스로서의 역할을 함
▪ 실제로 사용하는 데이터셋은 이를 상속하여 구현
▪ prepare 메서드가 데이터 준지 작업을 하도록 구현
▪ __getitem__ 메서드는 단순히 지정된 인덱스에 위치하는 데이터를 꺼냄
▪ __len__ 메서드는 데이터셋의 길이를 알려줌

<큰데이터셋의 경우>
▪ 작은 데이터셋은 Dataset 클래스의 인스턴스 변수인 data와 label에 직접 ndarray 인스턴스
를 유지해도 무리가 없음
▪ 큰 데이터셋의 구현 방식은 지금의 방식을 사용할 수 없음
▪ data 디렉터리와 label 디렉터리에 각가 100만 개의 데이터가 저장되어 있다고 가정
▪ 빅 데이터 처리 방법
• BigData 클래스를 초기화할 떄는 데이터를 읽지 않음
• 데이터에 접근하는 __getitem__(index)가 불리는 시점에 데이터를 읽음

In [ ]:
<신경망 입력 형태로의 데이터 준비>
▪ 신경망을 학습시킬 때는 데이터 셋 중 일부를 미니배치로 꺼냄
▪ 인덱스를 지정하여 batch에 여러 데이터가 리스트로 저장하고, ndarray 인스턴스로 변환
        (데이터를 DeZero 의 신경망에 입력하기 위해)
▪ batch 의 각 원소에서 데이터만을 꺼내 하나의 ndarray 인스턴스로 변형함(이어 붙임) 


In [ ]:
<Spiral 클래스를 사용하여 학습시 달라진 점>
    
▪ Spiral 클래스 사용시 미니배치를 만드는 부분의 코드를 수정
▪ Dataset 클래스를 사용하여 신경망을 학습
- 이점은 다른 데이터셋으로 교체해 학습할 때 확인할 수 있음
- Spiral을 BigData 로 교체하는 것만으로 훨씬 큰 데이터셋을 대응 할 수 있음
▪ 데이터셋 인터페이스를 통일하여 다양한 데이터셋을 똑같은 코드로 처리 할 수 있음

In [ ]:
<데이터셋 전처리>
    
▪ 모델에 데이터를 입력하기 전에 데이터를 특정한 형태로 가공할 경우가 많음
▪ 데이터 형상 변형, 데이터 확장 등등
▪ 초기화 시에 transform 과 target_transform 을 새롭게 받음
▪ transform 은 입력 데이터 하나에 대한 변환을 처리하고, target_transform 레이블 하나에
대한 변환을 처리함
( 이 값이 None 이면 전처리 로직은 lambda x : x 로 설정되어 받은 인수를 그대로 변환)
▪ 데이터를 스케일 변환 한다면 데이터셋에 사용자가 원하는 전처리를 추가할 수 있다.  